<a href="https://colab.research.google.com/github/Saaaaangmin/LLM/blob/main/LLM_PosToCat_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade openai
!pip install python-docx requests
!pip install pandas openpyxl requests

from fastapi import APIRouter, HTTPException
from pydantic import BaseModel
import openai
import pandas as pd
import os
from dotenv import load_dotenv

# 환경 변수 불러오기
load_dotenv()
openai.api_key = ''  # API 키 설정

# 로컬 경로 또는 Google Drive에서 Excel 데이터 불러오기
file_path = 'C:/Users/NICE/Desktop/OhSangmin/LLM/PosToCat/PosToCat_simplified_QA.xlsx'  # 경로에 맞게 조정
excel_data = pd.read_excel(file_path)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.9/386.9 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 4.1 MB/s eta 0:00:00


In [ ]:
router = APIRouter()

# 대화 이력을 저장할 리스트 생성
history = []

# 요청과 응답 데이터 모델
class ChatRequest(BaseModel):
    question: str

class ChatResponse(BaseModel):
    answer: str

# OpenAI API에 질문을 보내는 헬퍼 함수
def query_openai_api(question: str) -> str:
    excel_text = excel_data.to_csv(index=False)
    full_question = f"{question}\n\n{excel_text}\n이 데이터에 기반해서 설명해줘."

    # 현재 대화 이력을 포함하여 메시지 생성
    messages = [
        {"role": "system", "content": """너는 'Nice정보통신의 10년차 PosToCat 모듈 담당자야.
제공하는 파일은 CAT단말기 관련 정보 및 POS<->CAT 단말기 통신 및 VAN통신에 대해 설명되어있고, 파일 내 '질문' 열은 질문할 데이터가 있고, '응답' 열은 질문에 대한 대답이 작성되어있어.
파일의 내용을 살펴보고 이를 기반으로 답변을 줘야해. POS<->CAT단말기 간 연동 및 단말기 VAN통신에 대해 전문가처럼 답변해줘야 해.
모든 답변은 한국어로 해주고, 만약 헷갈리거나 정확한 답변이 아닐 경우 CS개발실 '02-2187-3638' 연락처로 연락해달라는 코멘트도 덧붙여줘."""}
    ]

    # 이전 대화 기록을 포함
    messages.extend(history)
    messages.append({"role": "user", "content": full_question})

    try:
        response = openai.ChatCompletion.create(
            model="gpt-4o",
            messages=messages,
            max_tokens=1000,
            temperature=0
        )
        answer = response.choices[0].message['content']

        # 새로운 질문/답변을 history에 추가
        history.append({"role": "user", "content": question})
        history.append({"role": "assistant", "content": answer})

        return answer
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

# JSON 형식의 질문을 받아 답변을 반환하는 API 엔드포인트
@router.post("/chat", response_model=ChatResponse)
async def chat_endpoint(request: ChatRequest):
    answer = query_openai_api(request.question)
    return {"answer": answer}